## Race, Class, and Student Debt as a Stratification System

### Research team: Charlie Eaton, Adam Goldstein, Laura Hamilton, Frederick Wherry

### Enrollment segregation IPEDS data exploration. Code by Charlie Eaton

### August 5, 2020

In [9]:
foreach year in 2018 {
copy https://nces.ed.gov/ipeds/datacenter/data/EF`year'A_Data_Stata.zip EF`year'A_Data_Stata.zip, replace
unzipfile EF`year'A_Data_Stata, replace
insheet using ef`year'a_data_stata.csv, clear
quietly egen efminort = rowtotal (ef2mort efaiant efnhpit efasiat), missing
foreach var in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
quietly gen ftfirstef`var'=ef`var' if line == 1
quietly gen ptfirstef`var'=ef`var' if line == 15
quietly gen allfirstef`var'=ef`var' if efalevel==4
quietly gen ftallef`var'=ef`var' if line == 14
quietly gen ptunderef`var'=ef`var' if line == 22
quietly gen ptprofef`var'=ef`var' if line == 23
quietly gen ptgradef`var'=ef`var' if line == 25
quietly gen ftunderef`var'=ef`var' if line == 8
quietly gen ftprofef`var'=ef`var' if line == 9
quietly gen ftgradef`var'=ef`var' if line == 11
}
**REPLACE ZERO VALUES WITH SMALL NEGATIVE VALUES**
foreach group in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
foreach level in ftfirst ptfirst allfirst ftall ptunder ptprof ptgrad ftunder ftprof ftgrad {
quietly bysort unitid `level'ef`group' : gen miss`level'ef`group' = missing(`level'ef`group'[1])
}
}
**COLLAPSE ENROLLMENT VARIABLES TO A SINGLE ROW FOR ALL INSTITUTIONS**
collapse (min) miss* (sum) ft* pt* all*, by(unitid)

foreach group in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
foreach level in ftfirst ptfirst allfirst ftall ptunder ptprof ptgrad ftunder ftprof ftgrad {
quietly replace `level'ef`group'=. if miss`level'ef`group'
}
}
    
drop miss* ptprof* ftprof*
save fteenrollment`year', replace

copy https://nces.ed.gov/ipeds/datacenter/data/HD`year'_Data_Stata.zip HD`year'_Data_Stata.zip, replace
unzipfile HD`year'_Data_Stata, replace
quietly insheet using hd`year'_data_stata.csv, clear
quietly sort unitid
quietly local m1=(`year'+1)
quietly save hd_`m1', replace

}

In [13]:
use hd_2019, clear
merge 1:1 unitid using fteenrollment2018, nogen
save hdfallenroll2019, replace




    Result                           # of obs.
    -----------------------------------------
    not matched                           505
        from master                       505  
        from using                          0  

    matched                             6,352  
    -----------------------------------------

file hdfallenroll2019.dta saved


In [14]:
ds, alpha varwidth(28) 

act               ftallefhispm      ftunderef2morm    ptfirsteftotlw
addr              ftallefhispt      ftunderef2mort    ptfirstefunknm
adminurl          ftallefhispw      ftunderef2morw    ptfirstefunknt
allfirstef2morm   ftallefminort     ftunderefaianm    ptfirstefunknw
allfirstef2mort   ftallefnhpim      ftunderefaiant    ptfirstefwhitm
allfirstef2morw   ftallefnhpit      ftunderefaianw    ptfirstefwhitt
allfirstefaianm   ftallefnhpiw      ftunderefasiam    ptfirstefwhitw
allfirstefaiant   ftallefnralm      ftunderefasiat    ptgradef2morm
allfirstefaianw   ftallefnralt      ftunderefasiaw    ptgradef2mort
allfirstefasiam   ftallefnralw      ftunderefbkaam    ptgradef2morw
allfirstefasiat   ftalleftotlm      ftunderefbkaat    ptgradefaianm
allfirstefasiaw   ftalleftotlt      ftunderefbkaaw    ptgradefaiant
allfirstefbkaam   ftalleftotlw      ftunderefhispm    ptgradefaianw
allfirstefbkaat   ftallefunknm      ftunderefhispt    ptgradefasiam
allfirstefbkaaw   ftallefunknt      ftund

In [11]:
%head

,unitid,ftfirsteftotlt,ptfirsteftotlt,allfirsteftotlt,ftalleftotlt,ptundereftotlt,ptgradeftotlt,ftundereftotlt,ftgradeftotlt,ftfirsteftotlm,ptfirsteftotlm,allfirsteftotlm,ftalleftotlm,ptundereftotlm,ptgradeftotlm,ftundereftotlm,ftgradeftotlm,ftfirsteftotlw,ptfirsteftotlw,allfirsteftotlw,ftalleftotlw,ptundereftotlw,ptgradeftotlw,ftundereftotlw,ftgradeftotlw,ftfirstefaiant,ptfirstefaiant,allfirstefaiant,ftallefaiant,ptunderefaiant,ptgradefaiant,ftunderefaiant,ftgradefaiant,ftfirstefaianm,ptfirstefaianm,allfirstefaianm,ftallefaianm,ptunderefaianm,ptgradefaianm,ftunderefaianm,ftgradefaianm,ftfirstefaianw,ptfirstefaianw,allfirstefaianw,ftallefaianw,ptunderefaianw,ptgradefaianw,ftunderefaianw,ftgradefaianw,ftfirstefasiat,ptfirstefasiat,allfirstefasiat,ftallefasiat,ptunderefasiat,ptgradefasiat,ftunderefasiat,ftgradefasiat,ftfirstefasiam,ptfirstefasiam,allfirstefasiam,ftallefasiam,ptunderefasiam,ptgradefasiam,ftunderefasiam,ftgradefasiam,ftfirstefasiaw,ptfirstefasiaw,allfirstefasiaw,ftallefasiaw,ptunderefasiaw,ptgradefasiaw,ftunderefasiaw,ftgradefasiaw,ftfirstefbkaat,ptfirstefbkaat,allfirstefbkaat,ftallefbkaat,ptunderefbkaat,ptgradefbkaat,ftunderefbkaat,ftgradefbkaat,ftfirstefbkaam,ptfirstefbkaam,allfirstefbkaam,ftallefbkaam,ptunderefbkaam,ptgradefbkaam,ftunderefbkaam,ftgradefbkaam,ftfirstefbkaaw,ptfirstefbkaaw,allfirstefbkaaw,ftallefbkaaw,ptunderefbkaaw,ptgradefbkaaw,ftunderefbkaaw,ftgradefbkaaw,ftfirstefhispt,ptfirstefhispt,allfirstefhispt,ftallefhispt,ptunderefhispt,ptgradefhispt,ftunderefhispt,ftgradefhispt,ftfirstefhispm,ptfirstefhispm,allfirstefhispm,ftallefhispm,ptunderefhispm,ptgradefhispm,ftunderefhispm,ftgradefhispm,ftfirstefhispw,ptfirstefhispw,allfirstefhispw,ftallefhispw,ptunderefhispw,ptgradefhispw,ftunderefhispw,ftgradefhispw,ftfirstefwhitt,ptfirstefwhitt,allfirstefwhitt,ftallefwhitt,ptunderefwhitt,ptgradefwhitt,ftunderefwhitt,ftgradefwhitt,ftfirstefwhitm,ptfirstefwhitm,allfirstefwhitm,ftallefwhitm,ptunderefwhitm,ptgradefwhitm,ftunderefwhitm,ftgradefwhitm,ftfirstefwhitw,ptfirstefwhitw,allfirstefwhitw,ftallefwhitw,ptunderefwhitw,ptgradefwhitw,ftunderefwhitw,ftgradefwhitw,ftfirstefunknt,ptfirstefunknt,allfirstefunknt,ftallefunknt,ptunderefunknt,ptgradefunknt,ftunderefunknt,ftgradefunknt,ftfirstefunknm,ptfirstefunknm,allfirstefunknm,ftallefunknm,ptunderefunknm,ptgradefunknm,ftunderefunknm,ftgradefunknm,ftfirstefunknw,ptfirstefunknw,allfirstefunknw,ftallefunknw,ptunderefunknw,ptgradefunknw,ftunderefunknw,ftgradefunknw,ftfirstef2mort,ptfirstef2mort,allfirstef2mort,ftallef2mort,ptunderef2mort,ptgradef2mort,ftunderef2mort,ftgradef2mort,ftfirstef2morm,ptfirstef2morm,allfirstef2morm,ftallef2morm,ptunderef2morm,ptgradef2morm,ftunderef2morm,ftgradef2morm,ftfirstef2morw,ptfirstef2morw,allfirstef2morw,ftallef2morw,ptunderef2morw,ptgradef2morw,ftunderef2morw,ftgradef2morw,ftfirstefnralt,ptfirstefnralt,allfirstefnralt,ftallefnralt,ptunderefnralt,ptgradefnralt,ftunderefnralt,ftgradefnralt,ftfirstefnralm,ptfirstefnralm,allfirstefnralm,ftallefnralm,ptunderefnralm,ptgradefnralm,ftunderefnralm,ftgradefnralm,ftfirstefnralw,ptfirstefnralw,allfirstefnralw,ftallefnralw,ptunderefnralw,ptgradefnralw,ftunderefnralw,ftgradefnralw,ftfirstefnhpit,ptfirstefnhpit,allfirstefnhpit,ftallefnhpit,ptunderefnhpit,ptgradefnhpit,ftunderefnhpit,ftgradefnhpit,ftfirstefnhpim,ptfirstefnhpim,allfirstefnhpim,ftallefnhpim,ptunderefnhpim,ptgradefnhpim,ftunderefnhpim,ftgradefnhpim,ftfirstefnhpiw,ptfirstefnhpiw,allfirstefnhpiw,ftallefnhpiw,ptunderefnhpiw,ptgradefnhpiw,ftunderefnhpiw,ftgradefnhpiw,ftfirstefminort,ptfirstefminort,allfirstefminort,ftallefminort,ptunderefminort,ptgradefminort,ftunderefminort,ftgradefminort
1,100654,1402,3,1405,5492,301,313,4842,650,571,1,572,2140,139,102,1967,173,831,2,833,3352,162,211,2875,477,3,0,3,12,1,2,11,1,2,0,2,3,0,1,3,0,1,0,1,9,1,1,8,1,2,0,2,9,2,5,7,2,1,0,1,5,0,3,4,1,1,0,1,4,2,2,3,1,1297,3,1300,4906,260,251,4424,482,522,1,523,1918,117,85,1785,133,775,2,777,2988,143,166,2639,349,11,0,11,44,5,0,41,3,6,0,6,22,3,0,22,0,5,0,5,22,2,0,19,3

## A. State level isolation measure

### Average percentage of students of other race at school for average student of a given race

In [41]:
quietly {
use hdfallenroll2019, clear
gen wxblackpctall=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt)
gen wxblackpct4yr=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt) if iclevel==1
gen wxblackpct2yr=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt) if iclevel==2

gen wxlatinpctall=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt)
gen wxlatinpct4yr=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt) if iclevel==1
gen wxlatinpct2yr=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt) if iclevel==2

gen bxwhitepctall=ftfirstefbkaat *(ftfirstefwhitt / ftfirsteftotlt)
gen bxwhitepct4yr=ftfirstefbkaat *(ftfirstefwhitt / ftfirsteftotlt) if iclevel==1
gen bxwhitepct2yr=ftfirstefbkaat *(ftfirstefwhitt / ftfirsteftotlt) if iclevel==2

gen lxwhitepctall=ftfirstefhispt *(ftfirstefwhitt / ftfirsteftotlt)
gen lxwhitepct4yr=ftfirstefhispt *(ftfirstefwhitt / ftfirsteftotlt) if iclevel==1
gen lxwhitepct2yr=ftfirstefhispt *(ftfirstefwhitt / ftfirsteftotlt) if iclevel==2

foreach var in ftfirstefbkaat ftfirstefhispt ftfirstefwhitt {
    gen `var'4yr =`var' if iclevel==1
    gen `var'2yr =`var' if iclevel==2
    rename `var' `var'all
}

collapse (sum) ftfirstefbkaat* ftfirstefhispt* ftfirstefwhitt* bx* lx* wx*, by(stabbr)

foreach level in all 4yr 2yr {
    gen blackiso`level'=100*(1/ftfirstefbkaat`level')*bxwhitepct`level'
    gen latiniso`level'=100*(1/ftfirstefhispt`level')*lxwhitepct`level'
    gen whiteblkiso`level'=100*(1/ftfirstefwhitt`level')*wxblackpct`level'
    gen whitelatiniso`level'=100*(1/ftfirstefwhitt`level')*wxlatinpct`level'
    }

foreach var in blackiso latiniso whiteblkiso whitelatiniso {
    gen `var'dif=`var'4yr - `var'2yr
    }
drop if stabbr=="PR" | stabbr=="GU" | stabbr=="VI" | stabbr=="PW" | stabbr=="AS" | stabbr=="MH" | ///
    stabbr=="MP" | stabbr=="FM"
}

In [45]:
set line 200
sort blackisodif

display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE WHITE AT SCHOOLS OF AVERAGE BLACK STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr blackiso*

sort latinisodif
display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE WHITE AT SCHOOLS OF AVERAGE LATINX STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr latiniso*

sort whiteblkisodif
display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE BLACK AT SCHOOLS OF AVERAGE WHITE STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr whiteblk*

sort whitelatinisodif
display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE LATINX AT SCHOOLS OF AVERAGE WHITE STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr whitelatin*




AVERAGE PERCENTAGE OF STUDENTS WHO ARE WHITE AT SCHOOLS OF AVERAGE BLACK STUDENTS

NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS


     +-----------------------------------------------------+
     | stabbr   blacki~l   blac~4yr   blac~2yr   blackis~f |
     |-----------------------------------------------------|
  1. |     NC   32.43131   27.85243   45.81307   -17.96063 |
  2. |     UT   39.16281   38.16956   53.60085   -15.43129 |
  3. |     AL   36.49926   31.30774   46.19272   -14.88498 |
  4. |     SC   41.82921   36.79178    51.5837   -14.79192 |
  5. |     NH   61.49035   60.94383   74.46963   -13.52579 |
     |-----------------------------------------------------|
  6. |     WV   73.21539   71.40115   84.77285    -13.3717 |
  7. |     DE   32.56473   32.58827   42.10526   -9.516994 |
  8. |     MS   38.32386   34.09443   43.39775   -9.303326 |
  9. |     VA   38.65405   36.57902   44.79848   -8.219467 |
 10. |     AR   45.19062   43.76178    50.8917   -7.129925 |
     |---------

## 1. Calculate % undergrads who are white, Black, Latinx, Asian, American Indian / Alaska Native
(I count part time students as equal to full time students)

In [32]:
use hdfallenroll2019, replace
foreach group in totlt bkaat hispt whitt asiat aiant {
quietly egen under`group'=rowtotal(ptunderef`group' ftunderef`group'), missing
quietly gen p_`group'=under`group' / undertotlt
}
quietly label var p_bkaat "Black %"
quietly label var p_hispt "Latinx %"
quietly label var p_whitt "white %"
quietly label var p_asiat "Asian %"
quietly label var p_aiant "American Indian %"

In [37]:
label define sector 1 "public 4-year" 2 "private 4-year" ///
3 "for-profit 4-year" 4 "community college" 5 "private 2-year" 6 "for-profit 2-year" ///
7 "public < 2-year" 8 "private < 2-year" 9 "for-profit < 2-year"
label val sector sector 
tabstat p_*, s(mean) by(sector)





Summary statistics: mean
  by categories of: sector (SECTOR)

          sector |   p_totlt   p_bkaat   p_hispt   p_whitt   p_asiat   p_aiant
-----------------+------------------------------------------------------------
               0 |         .         .         .         .         .         .
   public 4-year |         1  .1264146  .1543375  .5383507  .0491909  .0198451
  private 4-year |         1  .1297046  .1281487  .5552864  .0396717  .0082442
for-profit 4-yea |         1  .2355439  .2038118  .3404863  .0486755   .009386
community colleg |         1   .130737  .1775414  .5479382  .0350867  .0269191
  private 2-year |         1  .1773106  .1069667  .5741934  .0353598  .0227187
for-profit 2-yea |         1  .2240095  .2095547  .4415495  .0340971  .0100233
 public < 2-year |         1  .1473891  .0998981  .6648113  .0156243    .01929
private < 2-year |         1  .1896863  .3327325  .3454089   .059994  .0110619
for-profit < 2-y |         1  .2487909  .2428464  .4026203  .0355

## 2. The following models estimate the extent to which % of a school's students from race/ethnicity groups in 2019 are explained by the state they are in for major sectors:

In [56]:
quietly est clear
quietly eststo: regress p_whitt i.fips if iclevel==1 & control==1 // state
quietly eststo: regress p_whitt i.fips if iclevel==1 & control==2 // private
quietly eststo: regress p_whitt i.fips if control==3 // for-profit
quietly eststo: regress p_whitt i.fips if iclevel==2 & control==1 // community

esttab, keep( ) stats(r2 N, fmt(3 0 0)  ///
	label(R-square N))

quietly est clear
quietly eststo: regress p_bkaat i.fips if iclevel==1 & control==1 // state
quietly eststo: regress p_bkaat i.fips if iclevel==1 & control==2 // private
quietly eststo: regress p_bkaat i.fips if control==3 // for-profit
quietly eststo: regress p_bkaat i.fips if iclevel==2 & control==1 // community

esttab, keep( ) stats(r2 N, fmt(3 0 0)  ///
	label(R-square N))








----------------------------------------------------------------------------
                      (1)             (2)             (3)             (4)   
                  p_whitt         p_whitt         p_whitt         p_whitt   
----------------------------------------------------------------------------
----------------------------------------------------------------------------
R-square            0.489           0.353           0.407           0.467   
N                     769            1413            2466             962   
----------------------------------------------------------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001







----------------------------------------------------------------------------
                      (1)             (2)             (3)             (4)   
                  p_bkaat         p_bkaat         p_bkaat         p_bkaat   
----------------------------------------------------------------------------
-

## 3. The following models estimate the extent to which % of a school's students from race/ethnicity groups in 2019 are explained by the CBSA they are located in, within each major higher ed sectors:

My take on these models is that they suggest that spatial segregation is a massive driver of racial segregation -- especially within public institutions, including public 4-year schools. A school's CBSA explains:
* 78% of the variation in the share of students who are white at public universities
* 63% of the variation in the share of students who are black at public universities
* 82% of the variation in the share of students who are white at community colleges
* 73% of the variation in the share of students who are black at community colleges

CBSAs, which capture differences within states, explain much more variation that state indicators. This likely reflects that commuter schools provide the majority of bachelor degree education in the U.S.

We could replicate this for Latinx students. But we may want want to restrict that analysis to a subset of states because of the greater concentration of Latinx students in one particular region.

In [54]:
quietly est clear
replace cbsa=. if cbsa==-2
set matsize 2000
quietly eststo: regress p_whitt i.cbsa if iclevel==1 & control==1
quietly eststo: regress p_whitt i.cbsa if iclevel==1 & control==2
quietly eststo: regress p_whitt i.cbsa if control==3
quietly eststo: regress p_whitt i.cbsa if iclevel==2 & control==1

esttab, keep( ) stats(r2 N, fmt(3 0 0)  ///
	label(R-square N))



(0 real changes made)







----------------------------------------------------------------------------
                      (1)             (2)             (3)             (4)   
                  p_whitt         p_whitt         p_whitt         p_whitt   
----------------------------------------------------------------------------
----------------------------------------------------------------------------
R-square            0.780           0.524           0.601           0.818   
N                     723            1350            2443             838   
----------------------------------------------------------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


In [57]:
quietly est clear
set matsize 2000
quietly eststo: regress p_bkaat i.cbsa if iclevel==1 & control==1
quietly eststo: regress p_bkaat i.cbsa if iclevel==1 & control==2
quietly eststo: regress p_bkaat i.cbsa if control==3
quietly eststo: regress p_bkaat i.cbsa if iclevel==2 & control==1

esttab, keep( ) stats(r2 N, fmt(3 0 0)  ///
	label(R-square N))









----------------------------------------------------------------------------
                      (1)             (2)             (3)             (4)   
                  p_bkaat         p_bkaat         p_bkaat         p_bkaat   
----------------------------------------------------------------------------
----------------------------------------------------------------------------
R-square            0.633           0.459           0.526           0.732   
N                     723            1350            2443             838   
----------------------------------------------------------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


## 4. The following models estimate the extent to which % of a school's students from race/ethnicity groups in 2019 are explained by:
1. The state the school is in (i.fips state fixed effect)
2. The highest degree offered by the school
3. The control of the school (public/private/for-profit)

These show that an additional 3% of the variation in % students who are white is explained by control and highest degree offered by school.

An additional 6% of the variation in % students who are Black is explained by control and highest degree offered by school.

On my to do list, merging in additional theorized factors like:
    1. school's admissions rate

These models are an additional way to look at explanations of racial distribution of enrollments to the segregation measures used in the papers we have reviewed.

In [40]:
est clear

quietly eststo: regress p_whitt i.fips
quietly eststo: regress p_whitt i.fips ib3.iclevel
quietly eststo: regress p_whitt i.fips ib3.iclevel ib3.control

esttab, keep(1.iclevel 2.iclevel 1.control 2.control) stats(r2 N, fmt(3 0 0) ///
	label(R-square N))







------------------------------------------------------------
                      (1)             (2)             (3)   
                  p_whitt         p_whitt         p_whitt   
------------------------------------------------------------
1.iclevel                          0.0578***      -0.0253** 
                                   (8.17)         (-2.84)   

2.iclevel                          0.0530***     -0.00583   
                                   (6.84)         (-0.70)   

1.control                                           0.126***
                                                  (16.32)   

2.control                                           0.119***
                                                  (12.86)   
------------------------------------------------------------
R-square            0.366           0.374           0.403   
N                    6053            6053            6053   
------------------------------------------------------------
t statistics in

In [41]:
est clear

quietly eststo: regress p_bkaat i.fips
quietly eststo: regress p_bkaat i.fips ib3.iclevel
quietly eststo: regress p_bkaat i.fips ib3.iclevel ib3.control

esttab, keep(1.iclevel 2.iclevel 1.control 2.control) stats(r2 N, fmt(3 0 0) ///
	label(R-square N))







------------------------------------------------------------
                      (1)             (2)             (3)   
                  p_bkaat         p_bkaat         p_bkaat   
------------------------------------------------------------
1.iclevel                         -0.0834***      -0.0166*  
                                 (-14.47)         (-2.30)   

2.iclevel                         -0.0631***      -0.0137*  
                                 (-10.01)         (-2.02)   

1.control                                          -0.107***
                                                 (-16.98)   

2.control                                         -0.0938***
                                                 (-12.47)   
------------------------------------------------------------
R-square            0.277           0.302           0.336   
N                    6053            6053            6053   
------------------------------------------------------------
t statistics in

In [42]:
est clear

quietly eststo: regress p_hispt i.fips
quietly eststo: regress p_hispt i.fips ib3.iclevel
quietly eststo: regress p_hispt i.fips ib3.iclevel ib3.control

esttab, keep(1.iclevel 2.iclevel 1.control 2.control) stats(r2 N, fmt(3 0 0) ///
	label(R-square N))







------------------------------------------------------------
                      (1)             (2)             (3)   
                  p_hispt         p_hispt         p_hispt   
------------------------------------------------------------
1.iclevel                         -0.0570***      -0.0409***
                                 (-12.59)         (-7.03)   

2.iclevel                         -0.0167***      -0.0114*  
                                  (-3.37)         (-2.09)   

1.control                                        -0.00923   
                                                  (-1.82)   

2.control                                         -0.0281***
                                                  (-4.66)   
------------------------------------------------------------
R-square            0.597           0.608           0.610   
N                    6053            6053            6053   
------------------------------------------------------------
t statistics in

In [31]:
%help list

This front-end cannot display HTML help.